<a href="https://colab.research.google.com/github/monacofj/moeabench/blob/add-test/misc/benchmark_gallery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MoeaBench: Scientific Audit Confrontation
This notebook provides a visual synthesis of the **Analytical Ground Truth (v0.7.6)** against live MOEA results.

It demonstrates the jump in mathematical precision and identifies the geometric displacement of legacy truths.


In [ ]:
# 1. Environment Setup & Root Discovery
import os
import sys
import pandas as pd
import numpy as np

def discover_root():
    # Check Colab
    if 'google.colab' in str(get_ipython()):
        if not os.path.exists('moeabench'):
            !git clone -b add-test https://github.com/monacofj/moeabench.git
        %cd moeabench
        !git pull origin add-test
        !pip install -e .
        return os.getcwd()
    
    # Local: search for MoeaBench package anchor
    curr = os.getcwd()
    for _ in range(3):
        if os.path.exists(os.path.join(curr, "MoeaBench")):
            return curr
        curr = os.path.dirname(curr)
    return os.getcwd()

PROJECT_ROOT = discover_root()
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import MoeaBench as mb
from MoeaBench.core import SmartArray
print(f"Project Root: {PROJECT_ROOT}")
print(f"MoeaBench version: {mb.system.version()}")


## Visual Auditor Helper
Logic to resolve absolute paths for artifacts and overlay data series (Ground Truth vs Current MOEA).

In [ ]:
def audit_confrontation(mop_name, M=3):
    print(f"--- Analyzing {mop_name} (M={M}) ---")
    
    # 1. Setup Problem & Manager
    exp = mb.experiment()
    try:
        mop = getattr(mb.mops, mop_name)(M=M)
    except:
        mop = getattr(mb.mops, mop_name)(M=M, D=M-1)
    exp.mop = mop
    
    # 2. Load Ground Truth (v0.7.6)
    try:
        f_gt_pop = exp.optimal(n_points=1000)
        F_gt = f_gt_pop.objectives
    except Exception:
        gt_file = os.path.join(PROJECT_ROOT, "tests/ground_truth", f"{mop_name}_{M}_optimal.csv")
        F_gt = pd.read_csv(gt_file, header=None).values
            
    gt_series = SmartArray(F_gt, name="Analytical Ground Truth")
    
    # 3. Run Optimizer for Comparison
    print(f"[PROCESS] Running NSGA-III for high-fidelity verification...")
    algo = mb.moeas.NSGA3()
    gens = 1200 if mop_name in ["DTLZ3", "DTLZ6", "DTLZ9"] else 800
    pop = 400 if mop_name in ["DTLZ3", "DTLZ6", "DTLZ9"] else 200
    if mop_name == "DTLZ8": gens, pop = 1500, 400

    exp.moea = algo
    exp.run(generations=gens, population=pop)
    F_moea = exp.front()
    moea_series = SmartArray(F_moea, name=f"NSGA-III (Gen={gens}, Pop={pop})")

    # 4. PLOT
    mb.view.topo_shape(gt_series, moea_series, title=f"{mop_name} (M={M}): Scientific Alignment")

## 1. DTLZ Family Audits

In [ ]:
audit_confrontation("DTLZ1")

In [ ]:
audit_confrontation("DTLZ2")

In [ ]:
audit_confrontation("DTLZ3")

In [ ]:
audit_confrontation("DTLZ4")

In [ ]:
audit_confrontation("DTLZ5")

In [ ]:
audit_confrontation("DTLZ6")

In [ ]:
audit_confrontation("DTLZ7")

In [ ]:
audit_confrontation("DTLZ7")

In [ ]:
audit_confrontation("DTLZ8")

In [ ]:
audit_confrontation("DTLZ9")

## 2. DPF Family Audits

In [ ]:
audit_confrontation("DPF1")

In [ ]:
audit_confrontation("DPF2")

In [ ]:
audit_confrontation("DPF3")

In [ ]:
audit_confrontation("DPF4")

In [ ]:
audit_confrontation("DPF5")

In [ ]:
audit_confrontation("DPF5")